<a href="https://colab.research.google.com/github/tute3a/BusquedasLaborales/blob/main/Busquedas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re
from datetime import datetime

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

total = []
listalinks = []
listalinks_error = []
Hoy = datetime.today().strftime('%d-%m-%Y')

for pagina in range(150):         #Entra página por página
    if pagina > -1:             # Por si se corta en el medio
        #url = "https://www.zonajobs.com.ar/ofertas-de-empleo-argentina-pagina-" + str(pagina) + ".html?recientes=true"
        url = "https://www.zonajobs.com.ar/ofertas-de-empleo-publicacion-menor-a-15-dias-pagina-" + str(pagina) + ".html?recientes=true"
        driver.get(url)

        avisos = driver.find_element_by_class_name("aviso-no-sponsor")      #Lista de avisos
        links = avisos.find_elements_by_css_selector("a")

        for link in links:          #Busca el link de todos los avisos
            enlace = link.get_attribute('href')
            if enlace != None:                          # Por si no hay enlace dentro de los links que encuentra
                if enlace.find('/empleos/') > 0 and not enlace in listalinks:
                #Toma sólo los links de empleos
                #Links que no se hayan abierto antes
                    print(enlace)
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(enlace)              #En una nueva pestaña abre link por link
                    try:                            #Intenta los comandos conocidos (para evitar que falle)
                        try:                        #Comandos para Zonajobs
                            info = driver.find_element_by_class_name("aviso_specs").text
                            descripcion = driver.find_element_by_class_name("aviso_description").text
                            titulo = driver.find_element_by_class_name('aviso_title').text
                            empresa = driver.find_element_by_class_name('aviso_company').text

                            detalle = re.compile("\n(.*)").findall(info)
                            detalle = ((titulo, empresa, detalle[0], detalle[2].replace("Hoy", Hoy), detalle[4], detalle[6], detalle[8],
                            descripcion.replace("/n", "")))

                        except:                     #Comandos para Bumeran
                            try:
                                info_1 = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]').text
                            except:
                                info_1 = ('Error5','Error6')

                            try:
                                info_2 = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]').text
                            except:
                                info_2 = ('Error7','Error8','Error8.0')

                            try:
                                descripcion = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div[1]').text
                            except:
                                descripcion = 'Error3'

                            try:
                                empresa = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[1]/div').text
                            except:
                                empresa = ('Error9','Error10')

                            try:
                                info_1 = re.split("\n", info_1)
                            except:
                                info_1 = ('Error5','Error6')

                            try:
                                info_2 = re.split("\n", info_2)
                            except:
                                info_2 = ('Error7','Error8','Error8.0')

                            try:
                                empresa = re.split("\n", empresa)
                            except:
                                empresa = ('Error9','Error10')

                            detalle = ((empresa[0], empresa[1], info_1[1], info_1[0].replace("Hoy", Hoy), info_2[1], info_2[0],
                            info_2[2], descripcion))


                    except:                         #Salida si no encuentra ni ZonaJobs ni Bumeran
                        detalle = (('Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error'))
                        # Guardar el link
                        el_link = ((enlace))
                        listalinks_error.append(el_link)  # Agrega el link abierto a la lista de links
                        # Revisar si podemos guardar algo de info


                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    el_link = ((enlace))
                    listalinks.append(el_link)      #Agrega el link abierto a la lista de links
                    total.append(detalle)           #Agrega la info a total

        print(pagina)
        df = pd.DataFrame(total, columns=['Trabajo', 'Empresa', 'Ubicación', 'Fecha', 'Salario', 'Jornada laboral', 'Área','Descripción'])
        print(df)       #Guarda una vez por página
        df.to_csv(r'ZonaJobsv3 - 6 noviembre 2020.csv', index=False, header=True)
        df_error = pd.DataFrame(listalinks_error, columns=['Links con errores'])
        df_error.to_csv(r'ZJErrores.csv',
                  index=False, header=True)

df = pd.DataFrame(total,columns=['Trabajo','Empresa','Ubicación','Fecha','Salario','Jornada laboral','Área','Descripción'])
print(df)
driver.close()
df.to_csv (r'ZonaJobsv3 - 6 noviembre 2020.csv', index = False, header=True)

#Links con errores
df_error = pd.DataFrame(listalinks_error, columns=['Links con errores'])
df_error.to_csv(r'ZJErrores.csv',
          index=False, header=True)

print("___________ FIN ___________")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


https://www.zonajobs.com.ar/empleos/contador-publico-1904325.html?indiceAviso=0
https://www.zonajobs.com.ar/empleos/jefe-rrll-gabriel-nigro-1904073.html?indiceAviso=1
https://www.zonajobs.com.ar/empleos/asistente-start-up-1903636.html?indiceAviso=2
https://www.zonajobs.com.ar/empleos/comercial-inmobiliario-bawilsa-desarrollos-1904070.html?indiceAviso=3
https://www.zonajobs.com.ar/empleos/encargado-jr-de-deposito-1904324.html?indiceAviso=4
https://www.zonajobs.com.ar/empleos/ingeniero-de-procesos-treves-argentina-s.a.-1904069.html?indiceAviso=5
https://www.zonajobs.com.ar/empleos/servicio-tecnico-posventa-importante-empresa-nacional-1904065.html?indiceAviso=6
https://www.zonajobs.com.ar/empleos/sushiman-importante-cadena-gastronomica-1592247.html?indiceAviso=7
https://www.zonajobs.com.ar/empleos/coordinador-administrativo-financiero-contable-1904321.html?indiceAviso=8
https://www.zonajobs.com.ar/empleos/restaurant-manager-prestigiosa-cadena-gastronomica-umano-hr-1878529.html?indiceAviso

In [ ]:
df = pd.read_csv (r'ZJErrores.csv')
for enlace in df['Link con errores']:
                #Toma sólo los links de empleos
                #Links que no se hayan abierto antes
                    print(enlace)
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(enlace)              #En una nueva pestaña abre link por link
                    try:                            #Intenta los comandos conocidos (para evitar que falle)
                        try:                        #Comandos para Zonajobs
                            info = driver.find_element_by_class_name("aviso_specs").text
                            descripcion = driver.find_element_by_class_name("aviso_description").text
                            titulo = driver.find_element_by_class_name('aviso_title').text
                            empresa = driver.find_element_by_class_name('aviso_company').text

                            detalle = re.compile("\n(.*)").findall(info)
                            detalle = ((titulo, empresa, detalle[0], detalle[2].replace("Hoy", Hoy), detalle[4], detalle[6], detalle[8],
                            descripcion.replace("/n", "")))

                        except:                     #Comandos para Bumeran
                            try:
                                info_1 = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]').text
                            except:
                                info_1 = ('Error5','Error6')

                            try:
                                info_2 = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]').text
                            except:
                                info_2 = ('Error7','Error8','Error8.0')

                            try:
                                descripcion = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div[1]').text
                            except:
                                descripcion = 'Error3'

                            try:
                                empresa = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/div/div[1]/div[1]/div/div[1]/div').text
                            except:
                                empresa = ('Error9','Error10')

                            try:
                                info_1 = re.split("\n", info_1)
                            except:
                                info_1 = ('Error5','Error6')

                            try:
                                info_2 = re.split("\n", info_2)
                            except:
                                info_2 = ('Error7','Error8','Error8.0')

                            try:
                                empresa = re.split("\n", empresa)
                            except:
                                empresa = ('Error9','Error10')

                            detalle = ((empresa[0], empresa[1], info_1[1], info_1[0].replace("Hoy", Hoy), info_2[1], info_2[0],
                            info_2[2], descripcion))


                    except:                         #Salida si no encuentra ni ZonaJobs ni Bumeran
                        detalle = (('Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error'))
                        # Guardar el link
                        el_link = ((enlace))
                        listalinks_error.append(el_link)  # Agrega el link abierto a la lista de links
                        # Revisar si podemos guardar algo de info


                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    el_link = ((enlace))
                    listalinks.append(el_link)      #Agrega el link abierto a la lista de links
                    total.append(detalle)           #Agrega la info a total


df = pd.DataFrame(total,columns=['Trabajo','Empresa','Ubicación','Fecha','Salario','Jornada laboral','Área','Descripción'])
print(df)
driver.close()
df.to_csv (r'ZonaJobsv3 - 6 noviembre 2020 v2.csv', index = False, header=True)

#Links con errores
df_error = pd.DataFrame(listalinks_error, columns=['Links con errores'])
df_error.to_csv(r'ZJErrores v2.csv',
          index=False, header=True)

print("___________ FIN ___________")